<h3>Pre-processing<h3>
<h5>Kita hanya perlu hot one encoding untuk mengkategorikan daerah agar bisa dimengerti model<h5>

In [1]:
import pandas as pd


In [2]:
df = pd.read_csv("data/cleaned_yogya_hp.csv")

In [3]:
df['listing-location'].value_counts()


listing-location
Ngaglik         137
Depok            92
Kalasan          65
Ngemplak         57
Gamping          54
Sleman           51
Mlati            50
Godean           26
Purwomartani     25
CondongCatur     20
Berbah           19
Prambanan        13
Sayegan          10
Kaliurang         7
Cebongan          5
Moyudan           3
Jombor            2
Tempel            2
Sidoarum          1
Turi              1
Pakem             1
Caturtunggal      1
Name: count, dtype: int64

In [4]:
df = pd.get_dummies(df, columns=['listing-location'], drop_first=True)

In [5]:
df.head()

,price,bed,bath,carport,surface_area,building_area,listing-location_Caturtunggal,listing-location_Cebongan,listing-location_CondongCatur,listing-location_Depok,...,listing-location_Ngaglik,listing-location_Ngemplak,listing-location_Pakem,listing-location_Prambanan,listing-location_Purwomartani,listing-location_Sayegan,listing-location_Sidoarum,listing-location_Sleman,listing-location_Tempel,listing-location_Turi
0,1790000000,3.0,3.0,2.0,120.0,110.0,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
1,170000000,3.0,2.0,1.0,102.0,126.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,695000000,2.0,2.0,1.0,100.0,100.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,560000000,3.0,1.0,1.0,109.0,67.0,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4,200000000,2.0,1.0,1.0,60.0,30.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 642 entries, 0 to 641
Data columns (total 27 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   price                          642 non-null    int64  
 1   bed                            642 non-null    float64
 2   bath                           642 non-null    float64
 3   carport                        642 non-null    float64
 4   surface_area                   642 non-null    float64
 5   building_area                  642 non-null    float64
 6   listing-location_Caturtunggal  642 non-null    bool   
 7   listing-location_Cebongan      642 non-null    bool   
 8   listing-location_CondongCatur  642 non-null    bool   
 9   listing-location_Depok         642 non-null    bool   
 10  listing-location_Gamping       642 non-null    bool   
 11  listing-location_Godean        642 non-null    bool   
 12  listing-location_Jombor        642 non-null    boo

Kita sudah berhasil membuat semua paramter dimengerti model, waktunya ke Modelling

<h3>Modelling<h3>

In [25]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


In [8]:
#set seed
np.random.seed(42)

In [10]:
# Feature Engineering  
def create_features(df):
    """Create additional features that might be predictive"""
    df_new = df.copy()
    
    # Ratio features
    df_new['building_to_surface_ratio'] = df_new['building_area'] / df_new['surface_area']
    df_new['price_per_sqm_building'] = df_new['price'] / df_new['building_area']
    df_new['price_per_sqm_surface'] = df_new['price'] / df_new['surface_area']
    
    # Room density features
    df_new['total_rooms'] = df_new['bed'] + df_new['bath']
    df_new['room_density'] = df_new['total_rooms'] / df_new['building_area']
    
    return df_new

# Prepare data
df_features = create_features(df)
cols_to_include = [col for col in df_features.select_dtypes(include='number').columns if not col.startswith('location_')]
corr_matrix = df_features[cols_to_include].corr()
print(corr_matrix)


                              price       bed      bath   carport  \
price                      1.000000  0.589641  0.710486  0.267045   
bed                        0.589641  1.000000  0.695099  0.285481   
bath                       0.710486  0.695099  1.000000  0.338274   
carport                    0.267045  0.285481  0.338274  1.000000   
surface_area               0.613885  0.507710  0.471821  0.201339   
building_area              0.765642  0.676245  0.721186  0.208020   
building_to_surface_ratio  0.518844  0.510974  0.561576  0.124921   
price_per_sqm_building     0.528619  0.098980  0.201404  0.138434   
price_per_sqm_surface      0.896696  0.481657  0.636260  0.240401   
total_rooms                0.709955  0.909984  0.930622  0.340436   
room_density              -0.298961 -0.142527 -0.153849 -0.043297   

                           surface_area  building_area  \
price                          0.613885       0.765642   
bed                            0.507710       0.676245 

fitur room density tidak menunjukkan korelasi yg bagus jadi kita hapus

In [ ]:
df = df_features.drop('room_density', axis=1)


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 642 entries, 0 to 641
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   price                          642 non-null    int64  
 1   bed                            642 non-null    float64
 2   bath                           642 non-null    float64
 3   carport                        642 non-null    float64
 4   surface_area                   642 non-null    float64
 5   building_area                  642 non-null    float64
 6   listing-location_Caturtunggal  642 non-null    bool   
 7   listing-location_Cebongan      642 non-null    bool   
 8   listing-location_CondongCatur  642 non-null    bool   
 9   listing-location_Depok         642 non-null    bool   
 10  listing-location_Gamping       642 non-null    bool   
 11  listing-location_Godean        642 non-null    bool   
 12  listing-location_Jombor        642 non-null    boo

In [15]:
X = df.drop("price", axis=1)
y = df['price']

In [ ]:
# Data splitting for proper evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
print(f"\nTrain set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")


Train set: 513 samples
Test set: 129 samples


In [ ]:
class RealEstateRegressor:
    def __init__(self):
        self.models = {}
        self.best_model = None
        self.best_cv_score = np.inf  # We want lowest RMSE, not highest R²

    def prepare_models(self):
        """Define different regression models to try - with overfitting fixes"""
        self.models = {
            'Linear': Pipeline([
                ('scaler', StandardScaler()),
                ('regressor', LinearRegression())
            ]),
            'Ridge': Pipeline([
                ('scaler', StandardScaler()),
                ('regressor', Ridge(alpha=1.0))
            ]),
            'Lasso': Pipeline([
                ('scaler', StandardScaler()),
                ('regressor', Lasso(alpha=1000.0, max_iter=2000))
            ]),
            # FIX 1: Reduce polynomial degree from 2 to lower degree
            'Polynomial_Linear_Degree1': Pipeline([
                ('poly', PolynomialFeatures(degree=1, include_bias=False)),
                ('scaler', StandardScaler()),
                ('regressor', LinearRegression())
            ]),
            # FIX 2: Add strong regularization to polynomial features
            'Polynomial_Ridge_Strong': Pipeline([
                ('poly', PolynomialFeatures(degree=2, include_bias=False)),
                ('scaler', StandardScaler()),
                ('regressor', Ridge(alpha=10000.0))  # Much stronger regularization
            ]),
            # FIX 3: Use Lasso with polynomial features to automatically select features
            'Polynomial_Lasso': Pipeline([
                ('poly', PolynomialFeatures(degree=2, include_bias=False)),
                ('scaler', StandardScaler()),
                ('regressor', Lasso(alpha=5000.0, max_iter=3000))
            ]),
            # FIX 4: Use ElasticNet for balanced L1/L2 regularization
            'Polynomial_ElasticNet': Pipeline([
                ('poly', PolynomialFeatures(degree=2, include_bias=False)),
                ('scaler', StandardScaler()),
                ('regressor', ElasticNet(alpha=1000.0, l1_ratio=0.5, max_iter=3000))
            ]),
            # FIX 5: Add feature selection before polynomial expansion
            'Polynomial_SelectK': Pipeline([
                ('selector', SelectKBest(score_func=f_regression, k=10)),  # Select top 10 features first
                ('poly', PolynomialFeatures(degree=2, include_bias=False)),
                ('scaler', StandardScaler()),
                ('regressor', Ridge(alpha=1000.0))
            ]),
            'RandomForest': Pipeline([
                ('regressor', RandomForestRegressor(
                    n_estimators=100, 
                    max_depth=10,
                    min_samples_split=5,
                    min_samples_leaf=2,
                    random_state=42
                ))
            ]),
            'GradientBoosting': Pipeline([
                ('regressor', GradientBoostingRegressor(
                    n_estimators=100,
                    max_depth=6,
                    learning_rate=0.1,
                    random_state=42
                ))
            ])
        }

    def check_data_leakage(self, X_train, X_test, y_train, y_test):
        """Check for potential data leakage issues"""
        print("=== DATA LEAKAGE CHECK ===")
        print(f"Training set size: {X_train.shape}")
        print(f"Test set size: {X_test.shape}")
        print(f"Feature count: {X_train.shape[1]}")
        
        # Check if we have more features than samples (classic overfitting scenario)
        if X_train.shape[1] >= X_train.shape[0]:
            print("⚠️  WARNING: More features than training samples - high risk of overfitting!")
        
        # Check for duplicate rows between train and test
        train_test_overlap = pd.merge(
            pd.DataFrame(X_train), 
            pd.DataFrame(X_test), 
            how='inner'
        ).shape[0]
        
        if train_test_overlap > 0:
            print(f"⚠️  WARNING: {train_test_overlap} overlapping rows between train and test sets!")
        
        # Check target distribution
        print(f"Train target - Mean: {y_train.mean():.2f}, Std: {y_train.std():.2f}")
        print(f"Test target - Mean: {y_test.mean():.2f}, Std: {y_test.std():.2f}")
        
        return train_test_overlap == 0

    def evaluate_models_with_validation(self, X_train, y_train, X_test, y_test):
        """Enhanced evaluation with better overfitting detection"""
        # First check for data issues
        self.check_data_leakage(X_train, X_test, y_train, y_test)
        
        results = {}
        cv_folds = 5

        for name, model in self.models.items():
            try:
                print(f"\nEvaluating {name}...")
                
                # Cross-validation
                cv_scores = cross_val_score(model, X_train, y_train, cv=cv_folds, scoring='neg_mean_squared_error')
                cv_rmse_scores = np.sqrt(-cv_scores)

                # Fit model
                model.fit(X_train, y_train)

                # Training predictions
                y_train_pred = model.predict(X_train)
                train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
                train_mae = mean_absolute_error(y_train, y_train_pred)
                train_r2 = r2_score(y_train, y_train_pred)

                # Test predictions
                y_test_pred = model.predict(X_test)
                test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
                test_mae = mean_absolute_error(y_test, y_test_pred)
                test_r2 = r2_score(y_test, y_test_pred)

                # Calculate overfitting indicators
                rmse_gap = train_rmse - test_rmse if test_rmse > train_rmse else 0
                r2_gap = train_r2 - test_r2
                cv_test_gap = cv_rmse_scores.mean() - test_rmse

                results[name] = {
                    'cv_mean_rmse': cv_rmse_scores.mean(),
                    'cv_std_rmse': cv_rmse_scores.std(),
                    'train_rmse': train_rmse,
                    'train_mae': train_mae,
                    'train_r2': train_r2,
                    'test_rmse': test_rmse,
                    'test_mae': test_mae,
                    'test_r2': test_r2,
                    'rmse_gap': rmse_gap,
                    'r2_gap': r2_gap,
                    'cv_test_gap': cv_test_gap,
                    'cv_scores': cv_scores
                }

                # Enhanced reporting with overfitting detection
                print(f"CV RMSE = {cv_rmse_scores.mean():.0f} ± {cv_rmse_scores.std() * 2:.0f}")
                print(f"Train R² = {train_r2:.4f}, Test R² = {test_r2:.4f}, Gap = {r2_gap:.4f}")
                
                # Flag potential overfitting
                if train_rmse < 100 and train_r2 > 0.999:
                    print("🚨 OVERFITTING DETECTED: Perfect/near-perfect training performance")
                elif r2_gap > 0.1:
                    print(f"⚠️  Possible overfitting: R² gap = {r2_gap:.4f}")
                elif train_rmse == 0:
                    print("🚨 SEVERE OVERFITTING: Zero training error (memorization)")

                # Model selection with overfitting protection
                # Reject models with clear overfitting signals
                is_overfitted = (train_r2 > 0.999) or (r2_gap > 0.02) or (train_rmse < 1000)
                
                if not is_overfitted:
                    # Only consider non-overfitted models for selection
                    if cv_rmse_scores.mean() < self.best_cv_score:
                        self.best_cv_score = cv_rmse_scores.mean()
                        self.best_model = name
                else:
                    print(f"   ❌ {name} rejected due to overfitting")
                
                # Alternative: Gap-based selection (uncomment to use instead)
                # if not hasattr(self, 'best_gap'):
                #     self.best_gap = np.inf
                # if r2_gap < self.best_gap and test_r2 > 0.85:  # Only consider models with good performance
                #     self.best_gap = r2_gap
                #     self.best_model = name

            except Exception as e:
                print(f"Error with {name}: {e}")

        return results

    def feature_importance(self, X):
        """Get feature importance if available"""
        model = self.fitted_model

        if hasattr(model, 'feature_importances_'):
            # For tree-based models (RandomForest, GradientBoosting)
            importance = model.feature_importances_
            features = X.columns

        elif hasattr(model, 'named_steps'):
            # For linear models inside pipelines
            regressor = model.named_steps['regressor']

            if hasattr(regressor, 'coef_'):
                importance = np.abs(regressor.coef_)

                # Handle PolynomialFeatures if used
                if 'poly' in model.named_steps:
                    poly = model.named_steps['poly']
                    features = poly.get_feature_names_out(X.columns)
                else:
                    features = X.columns
            else:
                return None
        else:
            return None

        # Make sure lengths match
        if len(features) != len(importance):
            print("Length mismatch: Cannot generate feature importance.")
            return None

        return pd.DataFrame({
            'feature': features,
            'importance': importance
        }).sort_values('importance', ascending=False)
    
    def fit_best_model(self, X_train, y_train, X_test, y_test):
        """Fit the best performing model and show detailed results"""
        if self.best_model:
            print(f"\nFitting best model: {self.best_model}")
            self.fitted_model = self.models[self.best_model]
            self.fitted_model.fit(X_train, y_train)

            y_train_pred = self.fitted_model.predict(X_train)
            train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
            train_mae = mean_absolute_error(y_train, y_train_pred)
            train_r2 = r2_score(y_train, y_train_pred)

            y_test_pred = self.fitted_model.predict(X_test)
            test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
            test_mae = mean_absolute_error(y_test, y_test_pred)
            test_r2 = r2_score(y_test, y_test_pred)

            print(f"Training Performance:")
            print(f"RMSE: {train_rmse:,.0f}, MAE: {train_mae:,.0f}, R²: {train_r2:.4f}")
            print(f"Test Performance:")
            print(f"RMSE: {test_rmse:,.0f}, MAE: {test_mae:,.0f}, R²: {test_r2:.4f}")

            return y_test_pred
        else:
            print("No model was selected as best. Please run evaluate_models first.")
            return None


In [36]:
# Initialize and run the regressor
regressor = RealEstateRegressor()
regressor.prepare_models()

print("=== Model Evaluation ===")
results = regressor.evaluate_models_with_validation(X_train, y_train, X_test, y_test)

print(f"\n=== Best Model Training ===")
y_test_pred = regressor.fit_best_model(X_train, y_train, X_test, y_test)

# Comprehensive Results Summary
print(f"\n" + "="*100)
print(f"COMPREHENSIVE REGRESSION RESULTS SUMMARY")
print(f"="*100)

# Create results DataFrame for paper
results_df = pd.DataFrame()
for model_name, metrics in results.items():
    results_df = pd.concat([results_df, pd.DataFrame({
        'Model': [model_name],
        'CV_Mean_RMSE': [f"{metrics['cv_mean_rmse']:.0f}"],
        'CV_Std_RMSE': [f"{metrics['cv_std_rmse']:.0f}"],
        'Train_RMSE': [f"{metrics['train_rmse']:.0f}"],
        'Train_MAE': [f"{metrics['train_mae']:.0f}"],
        'Train_R²': [f"{metrics['train_r2']:.4f}"],
        'Test_RMSE': [f"{metrics['test_rmse']:.0f}"],
        'Test_MAE': [f"{metrics['test_mae']:.0f}"],
        'Test_R²': [f"{metrics['test_r2']:.4f}"]
    })], ignore_index=True)

# Sort by test R² performance (higher is better)
results_df['Test_R²_numeric'] = [float(x) for x in results_df['Test_R²']]
results_df = results_df.sort_values('Test_R²_numeric', ascending=False).drop('Test_R²_numeric', axis=1)

print("TABLE 1: Model Performance Comparison")
print("-" * 100)
print(results_df.to_string(index=False))


=== Model Evaluation ===
=== DATA LEAKAGE CHECK ===
Training set size: (513, 30)
Test set size: (129, 30)
Feature count: 30
Train target - Mean: 1115962962.96, Std: 674433582.22
Test target - Mean: 1057155038.74, Std: 651613171.86

Evaluating Linear...
CV RMSE = 105508619 ± 24111998
Train R² = 0.9787, Test R² = 0.9700, Gap = 0.0087

Evaluating Ridge...
CV RMSE = 105344133 ± 24289911
Train R² = 0.9786, Test R² = 0.9662, Gap = 0.0124

Evaluating Lasso...
CV RMSE = 105507381 ± 24113201
Train R² = 0.9787, Test R² = 0.9700, Gap = 0.0087

Evaluating Polynomial_Linear_Degree1...
CV RMSE = 105508619 ± 24111998
Train R² = 0.9787, Test R² = 0.9700, Gap = 0.0087

Evaluating Polynomial_Ridge_Strong...
CV RMSE = 354801380 ± 60265820
Train R² = 0.7692, Test R² = 0.7174, Gap = 0.0518
   ❌ Polynomial_Ridge_Strong rejected due to overfitting

Evaluating Polynomial_Lasso...
CV RMSE = 23403195 ± 33199577
Train R² = 1.0000, Test R² = 0.9697, Gap = 0.0302
   ❌ Polynomial_Lasso rejected due to overfitting



In [39]:
results_df.to_csv("data/perbandingan_hasil.csv")